In [11]:
import networkx as nx

nx_graph = nx.read_graphml("../data/graph_metric.graphml")

: 

In [4]:
from igraph import Graph

# load the graph froma GraphML file
graph = Graph.Read_GraphML("../data/graph_metric.graphml")

print(graph.summary())

/var/folders/kc/c74q_vx95dx4__wm8dtz6q540000gn/T/ipykernel_65682/3315175042.py:4: RuntimeWarning: Ignoring <key id="found"> because of a missing 'attr.type' attribute. at src/io/graphml.c:751
  graph = Graph.Read_GraphML("../data/graph_metric.graphml")
/var/folders/kc/c74q_vx95dx4__wm8dtz6q540000gn/T/ipykernel_65682/3315175042.py:4: RuntimeWarning: Ignoring <key id="id"> because of a missing 'attr.type' attribute. at src/io/graphml.c:751
  graph = Graph.Read_GraphML("../data/graph_metric.graphml")
/var/folders/kc/c74q_vx95dx4__wm8dtz6q540000gn/T/ipykernel_65682/3315175042.py:4: RuntimeWarning: Ignoring <key id="type"> because of a missing 'attr.type' attribute. at src/io/graphml.c:751
  graph = Graph.Read_GraphML("../data/graph_metric.graphml")
/var/folders/kc/c74q_vx95dx4__wm8dtz6q540000gn/T/ipykernel_65682/3315175042.py:4: RuntimeWarning: Ignoring <key id="version"> because of a missing 'attr.type' attribute. at src/io/graphml.c:751
  graph = Graph.Read_GraphML("../data/graph_metric.

IGRAPH D--- 59152712 178155040 -- 
+ attr: id (e)


## Explore node attributes

In [8]:
# print the first 10 vertex attributes
for v in graph.vs[:100000]:
    if v.attributes() != {}:
        print(f"Vertex ID: {v.index}, Attributes: {v.attributes()}")

In [10]:
node = graph.vs[0]
print(node.attributes().items())

dict_items([])


In [6]:
for e in graph.es[:10]:  # Access the first 10 edges
    print(f"Edge ID: {e.index}, Attributes: {e.attributes()}")

Edge ID: 0, Attributes: {'id': 'e0'}
Edge ID: 1, Attributes: {'id': 'e1'}
Edge ID: 2, Attributes: {'id': 'e2'}
Edge ID: 3, Attributes: {'id': 'e3'}
Edge ID: 4, Attributes: {'id': 'e4'}
Edge ID: 5, Attributes: {'id': 'e5'}
Edge ID: 6, Attributes: {'id': 'e6'}
Edge ID: 7, Attributes: {'id': 'e7'}
Edge ID: 8, Attributes: {'id': 'e8'}
Edge ID: 9, Attributes: {'id': 'e9'}
